# 🍱 공무원 맛집 추천 시스템 머신러닝 분류 모델 개선

이 노트북은 서울시 공무원 업무추진비 데이터를 기반으로 머신러닝 분류 모델을 학습하여 맛집 카테고리를 예측하고, 해당 카테고리에 속하는 추천 음식점 이름을 출력합니다.

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# 데이터 불러오기
df = pd.read_csv("GongPick\data\프렌차이즈_구추출_결과.csv", encoding='utf-8')

# 결측 처리
df['업종 중분류'] = df['업종 중분류'].fillna('기타')
df['구'] = df['구'].fillna('기타')

# 사용할 컬럼 정의
features = ['인원', '계절', '점저', '1인당비용', '업종 중분류', '구']
label = '사용장소'

# 결측값 제거
df_clean = df[features + [label]].dropna()

# 사용장소 기준 상위 100개만 필터링
top_places = df_clean[label].value_counts().nlargest(100).index
df_filtered = df_clean[df_clean[label].isin(top_places)]

# 특성과 타깃 분리
X = df_filtered[features]
y = df_filtered[label]

# 수치형/범주형 특성 정의
numeric_features = ['인원', '1인당비용']
categorical_features = ['계절', '점저', '업종 중분류', '구']

# 전처리 파이프라인
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# 전체 파이프라인 구성 (RandomForest 사용)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# 학습/테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
pipeline.fit(X_train, y_train)

# 평가
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


                  precision    recall  f1-score   support

           (주)강가       0.60      0.51      0.55        35
 (주)바닷가작은부엌 덕수궁점       0.17      0.14      0.15        64
          (주)아리연       0.17      0.20      0.18        46
    (주)참치공방 시청지점       0.31      0.15      0.20        34
           VIP참치       0.03      0.04      0.04        28
            강서면옥       0.02      0.02      0.02        58
              고담       0.54      0.60      0.57        42
             고담2       0.45      0.40      0.42        35
           고려삼계탕       0.20      0.21      0.21        66
          곤트란쉐리에       1.00      1.00      1.00        25
            곰국시집       0.12      0.20      0.15       123
            구이구이       0.13      0.12      0.13       189
        남도한식 정든님       0.00      0.00      0.00        22
        남도한식-정든님       0.02      0.02      0.02        60
            남산복집       0.12      0.08      0.10        25
            남포면옥       0.02      0.02      0.02        56
            대

In [13]:
y.head()

2      무교동낙지
3       남산복집
8     목포세발낙지
13      배재반점
15     완산골명가
Name: 사용장소, dtype: object

In [12]:
y.info()

<class 'pandas.core.series.Series'>
Index: 29029 entries, 2 to 70684
Series name: 사용장소
Non-Null Count  Dtype 
--------------  ----- 
29029 non-null  object
dtypes: object(1)
memory usage: 453.6+ KB


In [23]:
# 예측 예시 입력
example = pd.DataFrame([{
    '인원': 7,
    '계절': '여름',
    '점저': '저녁',
    '1인당비용': 12000,
    '업종 중분류' : '일식',
    '구': '중구'
}])

# 예측 및 확률 계산
predicted_probs = pipeline.predict_proba(example)
predicted_class_index = predicted_probs.argmax()
predicted_place = pipeline.classes_[predicted_class_index]
confidence = predicted_probs[0][predicted_class_index]

# 예측 결과 출력
print("예측된 사용장소:", predicted_place)
print(f"신뢰도: {confidence:.2%}")

# 원본 데이터에서 해당 장소의 업종 중분류 조회
match = df[df['사용장소'] == predicted_place]
if not match.empty:
    category = match['업종 중분류'].iloc[0]
    print("업종 중분류:", category)

    # 동일 업종 중분류 내 비슷한 장소 3개 추천
    similar_places = df[(df['업종 중분류'] == category) & (df['사용장소'] != predicted_place)]
    top_similars = similar_places['사용장소'].value_counts().head(3).index.tolist()

    print("비슷한 장소 추천:")
    for place in top_similars:
        print("-", place)
else:
    print("해당 사용장소 정보가 원본 데이터에 없습니다.")


예측된 사용장소: 우도일식
신뢰도: 58.00%
업종 중분류: 일식
비슷한 장소 추천:
- 브이아이피참치
- 배수사
- 브이아이피참치 뉴서울호텔점
